In [9]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from numba import jit
from datetime import timedelta
from tqdm.auto import tqdm
from backtest_utils import *
import matplotlib.dates as mdates
import copy

# 示例数据：预期收益和协方差矩阵



In [10]:
start_dt = '20190101'
end_dt = '20240701'
test_dt = '20200101'
vol_day = 90
trade_date = pd.read_csv('trade_dates.csv')
trade_date = trade_date['cal_date'].astype(str)
data = pd.read_csv('etf.csv')
data = data.sort_values(by = ['ts_code','trade_date'],ascending = True)
bond_rate = pd.read_csv('bond_rate.csv')
bond_rate = bond_rate.set_index('trade_date')
bond_rate = bond_rate['close']
bond_rate.index = pd.to_datetime(bond_rate.index,format = '%Y%m%d')
bond_rate = bond_rate.reindex(trade_date, method='ffill')



In [11]:
def calculate_daily_returns(leverage_list,df, date_stocks,date_weights,initial_capital, fee_rate,deviate_rate,bond_rate,trade_date):
    #date_stocks: 每天买入的股票 type字典
    #df 每天股市各股票基本信息

    net_value = initial_capital
    net_values = []  # 初始净值
    daily_returns = []
    expected_holdings = []
    actual_holdings = []
    position_change = []
    holdings_diffs = []
    stock_earnings = []
    stock_earning = []
    
    df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
    fixed_trade_date =pd.to_datetime(trade_date[0])
    df = df.set_index(["ts_code","trade_date"])
    df = df["pct_chg"]/100
    position = 0
    
    for date, stock_list in tqdm(date_stocks.items(), desc="计算日收益"):
            free_rate = float((bond_rate[bond_rate.index == date].iloc[0] / 25200))

            return_list = []
            #判断持仓是否为空
            if len(actual_holdings) == 0:
                actual_holdings = date_weights[date]

            #利用风险平价计算的持仓
            expected_holdings = date_weights[date]

            #持仓的日收益率    
            for stock in stock_list: 
                
                if (stock, date) in df.index:

                    return_ = float(df.loc[(stock, date)]- free_rate)
                    return_list.append(return_)

                else: 
                    return_ = 0 - free_rate
                    return_list.append(return_)



            multiplied_result =np.array(leverage_list) * np.array(return_list)
            free_rate_array = np.full_like(multiplied_result, free_rate)
            return_list = multiplied_result + free_rate_array
            return_list = return_list.tolist()
            
            
            if stock_earning is None or len(stock_earning) == 0:
                stock_earning = np.zeros(len(return_list))
            if not np.isfinite(net_value):
                raise ValueError("net value is not finite")

            if not np.all(np.isfinite(return_list)) or not np.all(np.isfinite(actual_holdings)):
                raise ValueError("Return list or actual holdings contain non-finite values")
                
            stock_earning += net_value * np.array(return_list) * np.array(actual_holdings)
            stock_earnings.append(stock_earning.copy())
            
            #判断是否调整持仓 0不换 1换
            for i in range(len(expected_holdings)):
                diff_holding = actual_holdings[i]-expected_holdings[i]
                if abs(diff_holding)<= deviate_rate and pd.to_datetime(date) <= fixed_trade_date :

                    position = 0

                else:
                    position = 1
                    fixed_trade_date = pd.to_datetime(date) + timedelta(days = 60)
                    break

            holdings_diff = [actual - expected for actual,expected in zip(actual_holdings,expected_holdings)]
            holdings_diffs.append(holdings_diff)
            
            
            if position == 1:
                #print(f"{date}需要调整持仓")
                position_change.append(date)
                #计算持仓比例变化与交易费用


                abs_list = list(map(abs, holdings_diff))
                transaction_cost = sum(abs_list)* net_value * fee_rate
                net_value -= transaction_cost
                actual_holdings = copy.deepcopy(expected_holdings)


                adjusted_returns = [1 + returns for returns in return_list]                            
                # 计算新的净值
                new_values = [net_value * actual * adjusted_return for actual,adjusted_return in zip(actual_holdings,adjusted_returns)]
                #print(f'改日净值为：{np.sum(new_values) - net_value}')
                returns = np.sum(new_values)/net_value - 1
                daily_returns.append(float(returns))
                net_value = np.sum(new_values)
                actual_holdings = new_values/np.sum(new_values) 
                net_values.append(net_value)

            elif position == 0 :
                #print(f"{date}不需要调整持仓")

                #计算
                adjusted_returns = [1 + returns for returns in return_list]     
                # 计算新的净值
                new_values = [net_value * actual * adjusted_return for actual,adjusted_return in zip(actual_holdings,adjusted_returns)]
                #print(np.sum(new_values) - net_value)
                returns = np.sum(new_values)/net_value - 1
                daily_returns.append(float(returns))
                net_value = np.sum(new_values)
                actual_holdings = new_values/np.sum(new_values)
                net_values.append(net_value)

            if len(leverage_list) != len(stock_list):
                position = 1
                
    return daily_returns,net_values,position_change,holdings_diffs,stock_earnings,stock_list

In [12]:
def calculate_vol(data,leverage_list,trade_date,vol_day): 
    data['trade_date'] = pd.to_datetime(data['trade_date'], format='%Y%m%d')
    codes = sorted(list(set(data['ts_code'])))
    print(codes)
    dates = trade_date
    data_stock = data['ts_code']
    data_date = pd.to_datetime(data['trade_date']) 
    
    for i,code in enumerate(codes):
        for date in dates:
            one_year_ago = str(pd.to_datetime(date) - timedelta(days= vol_day))
            mask = (data_stock == code) & (data_date >= one_year_ago) & (data_date < date)
            daily_returns = data.loc[mask,"pct_chg"].fillna(0)
        
            data.loc[(data_stock == code) & (data_date == date), 'pct_chg'] *= leverage_list[i]
                
    return data,codes

In [13]:
stock_list = ['510300.SH','510500.SH','513500.SH','513660.SH','511010.SH','511260.SH','518880.SH','510170.SH','159985.SZ']
stocks = stock_list
num_assets = len(stocks)  # 假设 stocks 是你的资产列表
initial_leverage = [1,1,1.5,1,2.5,2.5,1.5,1.5,1.5]
stocks_dict = {
    'stocks_1': ['518880.SH', '510170.SH','159985.SZ'],
    'stocks_2': ['510300.SH', '510500.SH', '513500.SH', '513660.SH', '510170.SH'],
    'stocks_3': ['518880.SH','159985.SZ', '511260.SH', '511010.SH'],
    'stocks_4': ['510300.SH', '510500.SH', '513500.SH', '513660.SH', '511260.SH', '511010.SH']
}



In [14]:
data1,codes = calculate_vol(data,initial_leverage,trade_date,vol_day)

_, _, date_stocks = supporting_data(data1, trade_date, stocks,vol_day)

results = four_quarters(data,trade_date,stocks_dict,vol_day)
four_quarter = {}
for keys,values in results.items():
     for stock,weights in values.items():
        if stock in four_quarter.keys():
            four_quarter[stock]+= 1/4 * np.array(weights)
        else:
            four_quarter[stock] = 1/4 * np.array(weights)

# 确保顺序一致
code_seq = list(date_stocks.values())[0]
sorted_data = {code: four_quarter[code] for code in code_seq}
four_quarter = sorted_data

date_weights =  {date:[] for date in trade_date}
for i,date in enumerate(trade_date):
    for stock,weights in four_quarter.items():
        date_weights[date].append(weights[i])

['159985.SZ', '510170.SH', '510300.SH', '510500.SH', '511010.SH', '511260.SH', '513500.SH', '513660.SH', '518880.SH']


  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

In [7]:
def sharpe_ratio(initial_leverage,data,date_stocks,date_weights,bond_rate,trade_date,stocks,stocks_dict):
    
    data1,codes = calculate_vol(data,initial_leverage,trade_date,vol_day)
    _, _, date_stocks = supporting_data(data1, trade_date, stocks,vol_day)

    results = four_quarters(data1,trade_date,stocks_dict,vol_day)
    four_quarter = {}
    for keys,values in results.items():
        for stock,weights in values.items():
            if stock in four_quarter.keys():
                four_quarter[stock]+= 1/4 * np.array(weights)
            else:
                four_quarter[stock] = 1/4 * np.array(weights)

    # 确保顺序一致
    code_seq = list(date_stocks.values())[0]
    sorted_data = {code: four_quarter[code] for code in code_seq}
    four_quarter = sorted_data

    date_weights =  {date:[] for date in trade_date}
    for i,date in enumerate(trade_date):
        for stock,weights in four_quarter.items():
            date_weights[date].append(weights[i])
        
    daily_returns,net_values,position_change,holdings_diffs,stock_earnings,stock_list = calculate_daily_returns(initial_leverage,data1,date_stocks,date_weights,1e6,0.00015,0.04,bond_rate,trade_date)
    daily_returns = pd.Series(daily_returns)
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    #sharpe_ratio = daily_returns.mean() / daily_returns.std()* np.sqrt(252)
    return cumulative_returns.iloc[-1]

def negative_sharpe_ratio(leverage,data,date_stocks,date_weights,bond_rate,trade_date,stocks,stocks_dict):
    return -sharpe_ratio(leverage,data,date_stocks,date_weights,bond_rate,trade_date,stocks,stocks_dict)


    
bounds = [(1, 10)] * num_assets

options = {'disp': False, 'maxiter': 200, 'ftol': 1e-10}
result = minimize(negative_sharpe_ratio, initial_leverage, args=(data,date_stocks,date_weights,bond_rate,trade_date,stocks,stocks_dict),
                  method='SLSQP', bounds=bounds, options=options)

optimal_leverage = result.x
max_sharpe_ratio = -result.fun

print("Optimal Leverages:", optimal_leverage)
print("Maximum Sharpe Ratio:", max_sharpe_ratio)

['159985.SZ', '510170.SH', '510300.SH', '510500.SH', '511010.SH', '511260.SH', '513500.SH', '513660.SH', '518880.SH']


  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

...

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

计算日收益:   0%|          | 0/971 [00:00<?, ?it/s]

C:\Users\qtx27\AppData\Local\Temp\ipykernel_27948\3906140145.py:60: RuntimeWarning: overflow encountered in multiply
  stock_earning += net_value * np.array(return_list) * np.array(actual_holdings)
C:\Users\qtx27\AppData\Local\Temp\ipykernel_27948\3906140145.py:60: RuntimeWarning: invalid value encountered in add
  stock_earning += net_value * np.array(return_list) * np.array(actual_holdings)
C:\Users\qtx27\AppData\Local\Temp\ipykernel_27948\3906140145.py:93: RuntimeWarning: overflow encountered in scalar multiply
  new_values = [net_value * actual * adjusted_return for actual,adjusted_return in zip(actual_holdings,adjusted_returns)]
C:\Users\qtx27\AppData\Local\Temp\ipykernel_27948\3906140145.py:98: RuntimeWarning: invalid value encountered in divide
  actual_holdings = new_values/np.sum(new_values)


ValueError: net value is not finite

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd

# 定义Sharpe比率函数，适应TensorFlow操作
def sharpe_ratio(leverage_numpy, data, date_stocks, date_weights, bond_rate, trade_date, stocks, stocks_dict, vol_day):
    # 使用 leverage_numpy 而不是 leverage
    data1, codes = calculate_vol(data, leverage_numpy, trade_date, vol_day)
    _, _, date_stocks = supporting_data(data1, trade_date, stocks, vol_day)

    results = four_quarters(data1, trade_date, stocks_dict, vol_day)
    four_quarter = {}
    for keys, values in results.items():
        for stock, weights in values.items():
            if stock in four_quarter.keys():
                four_quarter[stock] += 1/4 * np.array(weights)
            else:
                four_quarter[stock] = 1/4 * np.array(weights)

    code_seq = list(date_stocks.values())[0]
    sorted_data = {code: four_quarter[code] for code in code_seq}
    four_quarter = sorted_data

    date_weights = {date: [] for date in trade_date}
    for i, date in enumerate(trade_date):
        for stock, weights in four_quarter.items():
            date_weights[date].append(weights[i])
    
    daily_returns, _, _, _, _, _ = calculate_daily_returns(leverage_numpy, data1, date_stocks, date_weights, 1e6, 0.00015, 0.04, bond_rate, trade_date)
    daily_returns = pd.Series(daily_returns)
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    
    return cumulative_returns.iloc[-1]

def negative_sharpe_ratio(leverage_numpy, data, date_stocks, date_weights, bond_rate, trade_date, stocks, stocks_dict, vol_day):
    return -sharpe_ratio(leverage_numpy, data, date_stocks, date_weights, bond_rate, trade_date, stocks, stocks_dict, vol_day)

def train_step():
    leverage_numpy = initial_leverage.numpy()
    with tf.GradientTape() as tape:
        loss = negative_sharpe_ratio(leverage_numpy, data, date_stocks, date_weights, bond_rate, trade_date, stocks, stocks_dict, vol_day)
    gradients = tape.gradient(loss, [initial_leverage])
    optimizer.apply_gradients(zip(gradients, [initial_leverage]))
    return loss

# 初始化变量
num_assets = 10  # 假设有10个资产，你需要根据实际情况调整
initial_leverage = [1.0] * num_assets
initial_leverage = tf.Variable(initial_leverage, dtype=tf.float32)
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# 假设这些数据已经初始化：data, date_stocks, date_weights, bond_rate, trade_date, stocks, stocks_dict, vol_day

# 训练
for i in range(200):  # 迭代次数
    loss = train_step()
    if i % 10 == 0:
        print(f"Step {i}, Loss: {loss.numpy()}")

optimal_leverage = initial_leverage.numpy()
max_sharpe_ratio = -loss.numpy()

print("Optimal Leverages:", optimal_leverage)
print("Maximum Sharpe Ratio:", max_sharpe_ratio)

['159985.SZ', '510170.SH', '510300.SH', '510500.SH', '511010.SH', '511260.SH', '513500.SH', '513660.SH', '518880.SH']


  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

  0%|          | 0/971 [00:00<?, ?it/s]

计算日收益:   0%|          | 0/971 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (10,) (9,) 